In [1]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script calculates current density maps starting from electric magnitude
# maps (Emag) generated using ROAST, for each of the 3 datasets included in the analysis
# (namely wp1a, wp1b and wp2a).
# Results are saved in 4d nifti files (1 volume per subject). For convenience, electric 
# field maps and masks are also saved in 4d nifti files. All results are smoothed with a
# FWHM (4mm kernel) before being saved. 

# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import glob
import numpy as np
from nilearn import image, plotting
from nilearn.image import new_img_like
from scipy import ndimage
import scipy.io
import time

# Function to calculate current density
from custom_functions.maps_functions import current_density_efield

In [2]:
## Parameters and variables: 
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' 
output_folder = 'D:\\roast-chapter3\\wp_all_results\\' # where to save results
data_folder = 'D:\\roast-chapter3\\'

# Tissue conductivities:
conductivities = [0.126, 0.276, 1.65] #WM, GM and CSF
fwhm_k = 4 # FWHM kernel used to smooth current density maps

# Datasets names and subjects lists
db_names = ['wp2a', 'wp1a', 'wp1b']
db_subjects = [['01','02','03','04','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','22','23','24'], #Wp2a
               ['03','04','05','07','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26'], # Wp1a
               ['01','02','03','04','05','06','07','08','09','10','11','12','13','15','16','17','18','19','21','22','23']] #Wp1b                              

## Loading AAL3 atlas and extracting M1 / Thalamus ROIs (regions of interest)
# AAL3 atlas paper:
# https://www.oxcns.org/papers/607%20Rolls%20Huang%20Lin%20Feng%20Joliot%202020%20AAL3.pdf 
AAl3_path = os.path.join(main_folder, 'rois', 'AAL3v1_1mm.nii')
AAL3_atlas = image.load_img(AAl3_path)

## Creating M1, Th and cerebellar masks from the AAL3 atlas. Load MNI template.
# AAL3 index for left M1 = 1
m1 = image.math_img("np.where(img == 1, 1, 0)", img = AAL3_atlas) 
# AAL3 index for TH = 121 - 149 (odd values only (left thalamus)) --> I'm not convinced about this one, ask Davinia
th = image.math_img("np.where(np.isin(img, np.arange(121, 150, 2)), 1, 0)", img = AAL3_atlas) 
# AAL3 index for right CB (102 and 108: cerebellar lobes IV-V and VIII)
cb = image.math_img("np.where(np.isin(img, np.array([102, 108])), 1, 0)", img = AAL3_atlas)
# MNI template for plotting and masking purposes
bg_img_map = image.load_img(os.path.join(main_folder, 'rois', 'MNI152_T1_1mm_Brain.nii'))

In [10]:
for db_id, db in enumerate(db_names): # Iterate all three datasets
    db_path = os.path.join(data_folder, db) # dataset folder
    
    # For each subject, I save the emag map, the brain mask, and the current density map
    shape = np.append(np.array([157, 189, 156]), len(db_subjects[db_id])) # shape of maps x N subjects
    emag_maps_array = np.zeros(shape)
    cd_maps_array = np.zeros(shape)
    masks_maps_array = np.zeros(shape)

    for i, subject in enumerate(db_subjects[db_id]): #Iterate all subjects
        path = db_path + '\sub-' + subject # Subject folder

        # Loading normalised Electric field magnitude map (emag) (unit: V/m) (e.g. wsub-*_emag.nii)
        # NB: The map was resampled and normalised with wp*_roast_3_post_roast_preprocessing.m
        emag_map = image.load_img(glob.glob(path + '\w*_emag.nii'))

        # The above scan (and all the other ones that will be loaded) has 4 dimensions,
        # with the 4th one referring to the number of volumes. However, these are 3d scans,
        # I therefore drop the 4th dimension (i.e. from (157, 189, 156, 1)) to
        # (157, 189, 156). This is handy when performing mathematical operations on the scans.
        scan_shape = emag_map.get_fdata().shape[0:3] # Shape of the scan (should be = (157, 189, 156))    
        emag_data = emag_map.get_fdata().reshape(scan_shape) # Data with 4th dimention in the array dropped
        emag_map = new_img_like(emag_map, emag_data) # Storing the data into a nibabel object

        # Resampling MNI anatomical file and ROIs to emag map (so that they have the same shape).
        # This is done only once as the shape is the same for every participant's scan.
        if not 'mni_resampled' in locals():
            mni_resampled = image.resample_to_img(bg_img_map, emag_map, interpolation = 'nearest')
        if not 'm1_resampled' in locals():
            m1_resampled = image.resample_to_img(m1, emag_map, interpolation = 'nearest')
            th_resampled = image.resample_to_img(th, emag_map, interpolation = 'nearest')
            cb_resampled = image.resample_to_img(cb, emag_map, interpolation = 'nearest')
    
        # Loading mask containing WM (index 1), GM (2), CSF (3), bone (4), skin (5), air (6).
        # e.g. wsub-*T1*T2_masks.nii.
        # This map was created by Roast starting from masks segmented by SPM. ROAST then
        # uses morphological operations followed by simple heuristics to fill remaining holes
        # (Huang et al., 2018).
        # ROAST scripts that create the mask: binaryMaskGenerate.m and segTouchup.m.
        # NB: The mask was resampled and normalised with wp2a_roast_3_post_roast_preprocessing.m
        mask_touched_all_data = image.load_img(glob.glob(path + '/w*_masks.nii')).get_fdata().reshape(scan_shape) 
        mask_touched_all = new_img_like(emag_map, mask_touched_all_data)
        
        # Excluding from the mask everything that is not WM, GM and CSF (idx 1, 2 and 3 respectively)
        mask_touched = image.math_img("np.where(np.isin(img, np.arange(1, 4)), img, 0)", img = mask_touched_all) 
            
        # Excluding from mask_touched values that outside the brain using the MNI template
        # NB: I do this because the grey matter map of SPM has some values that are outside the brain.
        mask_touched_brain = image.math_img("np.where(img2 != 0, img, 0)", img = mask_touched, img2 = mni_resampled) 
        
        # Masking emag map using the mask_touched_brain mask, to keep only WM, GM and CSF values
        emag_map_brain = image.math_img("np.where(img2 != 0, img, 0)", img = emag_map, img2 = mask_touched_brain)
        
        ## Computing current density (J) for each tissue (WM, GM, CSF), using the electric field (E) and tissue conductivity (s)
        # of each voxel, in accordance with Ohm’s law: J = sE. For more info: help(current_density_efield)    
        current_density = current_density_efield(emag_map_brain.get_fdata(), mask_touched_brain.get_fdata(), conductivities)
        current_density = new_img_like(emag_map, current_density) # Restoring data into nibabel object
        
        # Smoothing current density map with FWHM (4mm)
        current_density_sw = image.smooth_img(current_density, fwhm = fwhm_k)   
        
        # Smoothing emag map with FWHM (4mm). This is just for plotting purposes.
        emag_map_sw = image.smooth_img(emag_map_brain, fwhm = fwhm_k)

        # Lists containing electric field maps (smoothed), current density maps (smoothed) and
        # brain masks used for the calculation of the current density   
        emag_maps_array[:,:,:,i] = emag_map_sw.get_fdata()
        cd_maps_array[:,:,:,i] = current_density_sw.get_fdata()
        masks_maps_array[:,:,:,i] = mask_touched_brain.get_fdata()

    # Saving results (i.e. wp1a_all_emag_maps.nii)
    emag_tosave = new_img_like(emag_map, emag_maps_array).to_filename(os.path.join(output_folder, db + '_all_emag_maps.nii')) 
    cd_tosave = new_img_like(emag_map, cd_maps_array).to_filename(os.path.join(output_folder, db + '_all_cd_maps.nii'))
    masks_tosave = new_img_like(emag_map, masks_maps_array).to_filename(os.path.join(output_folder, db + '_all_masks_maps.nii'))